In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Function to generate noisy gravitational wave-like data
def generate_gw_data(n_samples, noise_level=0.1):
    time = np.linspace(0, 1, n_samples)
    signal = np.sin(2 * np.pi * time * 5)  # Simulated GW signal (sine wave)
    noise = noise_level * np.random.randn(n_samples)  # Adding Gaussian noise
    data = signal + noise
    return data, signal

# Prepare the data
n_samples = 100
x_data, y_data = generate_gw_data(n_samples)

# Reshape data for the neural network
x_data = x_data.reshape(-1, 1)
y_data = y_data.reshape(-1, 1)

# Build the neural network model
model = models.Sequential([
    layers.Dense(5, activation='relu', input_shape=(1,)),  # First hidden layer
    layers.Dense(5, activation='relu'),                    # Second hidden layer
    layers.Dense(1)                                        # Output layer
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),  # Learning rate = 0.1
              loss='mean_squared_error')

# Train the model
history = model.fit(x_data, y_data, epochs=100, verbose=1)

# Save the training loss plot
plt.figure()
plt.plot(history.history['loss'], label='Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('training_loss.png')  # Save figure
plt.show()

# Predict to compare true vs predicted signals
y_pred = model.predict(x_data)

# Save the true vs predicted signal plot
plt.figure()
plt.plot(x_data, y_data, label='True Signal', color='blue')
plt.plot(x_data, y_pred, label='Predicted Signal', color='orange', linestyle='dashed')
plt.legend()
plt.title('Gravitational Wave Signal: True vs Predicted')
plt.savefig('true_vs_predicted_signal.png')  # Save figure
plt.show()

# Synthetic data generation for the next model
A = 1   # Amplitude
f = 10  # Frequency in Hz
t = np.linspace(0, 1, 1000)  # Time array (1 second at 1000 samples/second)
y_true = A * np.sin(2 * np.pi * f * t)  # True GW signal
noise = 0.2 * np.random.normal(size=t.shape)  # Gaussian noise
X_test = y_true + noise  # Noisy input signal for testing

# Use the same data for training in this example
X_train = X_test
y_train = y_true

# Build a new neural network model
model = models.Sequential([
    layers.Dense(128, input_shape=(1,), activation='relu'),  # 128 neurons in the first hidden layer
    layers.Dense(64, activation='relu'),                      # 64 neurons in the second hidden layer
    layers.Dense(1, activation='linear')                      # Output layer for regression
])

# Compile the new model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the new model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Predict on the test data
predicted_signal = model.predict(X_test)

2024-10-24 21:14:29.374652: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Function to generate noisy gravitational wave-like data
def generate_gw_data(n_samples, noise_level=0.1):
    time = np.linspace(0, 1, n_samples)
    signal = np.sin(2 * np.pi * time * 5)  # Simulated GW signal (sine wave)
    noise = noise_level * np.random.randn(n_samples)  # Adding Gaussian noise
    data = signal + noise
    return data, signal

# Set the number of samples
n_samples = 100

# Prepare the data
x_data, y_data = generate_gw_data(n_samples)
x_data = x_data.reshape(-1, 1)  # Reshape for input shape (n_samples, 1)
y_data = y_data.reshape(-1, 1)  # Reshape to ensure it's (n_samples, 1)

print("x_data shape:", x_data.shape)
print("y_data shape:", y_data.shape)


# Build and compile the model
model = models.Sequential([
    layers.Dense(5, activation='relu', input_shape=(1,)),  # First hidden layer
    layers.Dense(5, activation='relu'),                    # Second hidden layer
    layers.Dense(1)                                        # Output layer
])
optimizer_01 = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=optimizer_01, loss='mean_squared_error')
print("Is model compiled?:", model.optimizer is not None)
print("Predictions shape:", model(x_data).shape)
print("y_data shape:", y_data.shape)

with tf.GradientTape() as tape:
    predictions = model(x_data)
    loss_value = tf.keras.losses.mean_squared_error(y_data, predictions)
    loss_value = tf.reduce_mean(loss_value)  # Ensure it is scalar
    
print("Loss value:", loss_value.numpy())


# Test with a very simple dataset
x_data_test = np.array([[1.0], [2.0], [3.0]])
y_data_test = np.array([[2.0], [4.0], [6.0]])

print("x_data_test shape:", x_data_test.shape)
print("y_data_test shape:", y_data_test.shape)

# Test with a single training step
with tf.GradientTape() as tape:
    predictions = model(x_data_test)
    loss_value = tf.keras.losses.mean_squared_error(y_data_test, predictions)
    loss_value = tf.reduce_mean(loss_value)
    print("Test loss value:", loss_value.numpy())

print(tf.__version__)


def extract_model_info(model, x_data, y_data, optimizer):
    info = {"step": [], "weights": [], "gradients": [], "outputs": [], "loss": []}

    for step in range(10):  # Run 10 steps of optimization
        with tf.GradientTape() as tape:
            predictions = model(x_data)
            # Explicitly calculate the loss using mean_squared_error
            loss_value = tf.keras.losses.mean_squared_error(y_data, predictions)
            loss_value = tf.reduce_mean(loss_value)  # Reduce to scalar

        # Compute gradients and get weights
        gradients = tape.gradient(loss_value, model.trainable_variables)
        weights = model.trainable_variables

        # Apply gradients (this updates the weights)
        optimizer.apply_gradients(zip(gradients, weights))

        # Store step information
        info["step"].append(step + 1)
        info["weights"].append([w.numpy() for w in weights])
        info["gradients"].append([g.numpy() for g in gradients])
        info["outputs"].append(predictions.numpy())
        info["loss"].append(loss_value.numpy())

        print(f"Step {step + 1}: Loss = {loss_value.numpy()}")

    return info


# Create optimizer for the two learning rates
optimizer_02 = tf.keras.optimizers.SGD(learning_rate=0.2)

# Extract information for both learning rates
info_lr_02 = extract_model_info(model, x_data, y_data, optimizer_02)
info_lr_01 = extract_model_info(model, x_data, y_data, optimizer_01)

# Convert extracted information into a table (e.g., using pandas DataFrame)
import pandas as pd

# Function to create a table showing weights, gradients, outputs, and loss for each step
def create_table(info):
    table = pd.DataFrame({
        'Step': info["step"],
        'Loss': info["loss"],  # Single float
        'Weights (Layer 1)': [str(w[0]) for w in info["weights"]],  # Array
        'Gradients (Layer 1)': [str(g[0]) for g in info["gradients"]],  # Array
        'Outputs': [str(o) for o in info["outputs"]]  # Array
    })
    return table

# Create tables for both learning rates
table_lr_01 = create_table(info_lr_01)
table_lr_02 = create_table(info_lr_02)

# Save the tables to CSV files
table_lr_01.to_csv('table_lr_01.csv', index=False)
table_lr_02.to_csv('table_lr_02.csv', index=False)

print("Tables saved successfully!")

# Visualizing the tables
print(table_lr_01.head())
print(table_lr_02.head())

In [1]:
# Save the true vs predicted signal plot for the new model
plt.figure(figsize=(10, 6))
plt.plot(t, y_true, label='True Signal', color="blue", linestyle="solid")
plt.plot(t, predicted_signal, label="Predicted Signal", color="orange", linestyle="dashed")
plt.xlabel("Time [s]")
plt.ylabel("Amplitude")
plt.title("Gravitational Wave Signal: True vs Predicted")
plt.legend()
plt.savefig('true_vs_predicted_signal_new_model.png')  # Save figure
plt.show()

# Save the loss over epochs for the new model
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Loss over Epochs', color='blue')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Model Loss Over Time")
plt.legend()
plt.savefig('loss_over_epochs_new_model.png')  # Save figure
plt.show()

# If there is a new noisy signal, predict with the new model
# Ensure to define 'noisy_signal' if using real or different data
# Uncomment below if you have 'noisy_signal' data to predict on:
predicted_signal = model.predict(noisy_signal)

# Save the final true vs predicted signal plot
# Uncomment below if you have 'true_signal' and 'predicted_signal' for a new prediction:
plt.figure(figsize=(10, 6))
plt.plot(t, true_signal, label="True Signal", color="blue")
plt.plot(t, predicted_signal, label="Predicted Signal", color="orange", linestyle="dashed")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.title("Gravitational Wave Signal: True vs Predicted (Time Series)")
plt.legend()
plt.savefig('final_true_vs_predicted_signal.png')  # Save figure
plt.show()

NameError: name 'plt' is not defined

In [11]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

AttributeError: module 'pydot' has no attribute 'InvocationException'

In [37]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install graphviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
from tabulate import tabulate

# Load CSV file (you can replace the file path with your actual CSV file)
df = pd.read_csv('your_csv_file.csv')

# Print table with headers for the CSV data
# Rename columns as per your needs to match your example
df.columns = ['step', 'w1', 'b1', 'z1', 'y1', '∇L(w1/b1)', 'w2', 'b2', 'z2', 'y2', '∇L(w2)', '∇L(b2)', 'L']

# Display in a tabulated format similar to the example table
print(tabulate(df, headers='keys', tablefmt='fancy_grid'))


ModuleNotFoundError: No module named 'tabulate'